In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,split

In [22]:
spark=SparkSession.builder.appName("top_3_maximum_values_PM10").getOrCreate()

In [23]:
df=spark.read.text("data.txt")
df.show()

+------------------+
|             value|
+------------------+
|s1,2016-01-01,20.5|
|s2,2016-01-01,30.1|
|s1,2016-01-02,60.2|
|s2,2016-01-02,20.4|
|s1,2016-01-03,55.5|
|s2,2016-01-03,52.5|
+------------------+



In [24]:
df = df \
    .withColumn("sensor", split(col("value"),",")[0]) \
    .withColumn("years", split(split(col("value"),",")[1],"-")[0]) \
    .withColumn("month", split(split(col("value"),",")[1],"-")[1]) \
    .withColumn("day", split(split(col("value"),",")[1],"-")[2]) \
    .withColumn("score",  split(col("value"),",")[2]) \
    .drop("value") 
df.show(truncate=False)

+------+-----+-----+---+-----+
|sensor|years|month|day|score|
+------+-----+-----+---+-----+
|s1    |2016 |01   |01 |20.5 |
|s2    |2016 |01   |01 |30.1 |
|s1    |2016 |01   |02 |60.2 |
|s2    |2016 |01   |02 |20.4 |
|s1    |2016 |01   |03 |55.5 |
|s2    |2016 |01   |03 |52.5 |
+------+-----+-----+---+-----+



In [25]:
score_sorted=df.sort("score", ascending=False)
score_sorted.show()

+------+-----+-----+---+-----+
|sensor|years|month|day|score|
+------+-----+-----+---+-----+
|    s1| 2016|   01| 02| 60.2|
|    s1| 2016|   01| 03| 55.5|
|    s2| 2016|   01| 03| 52.5|
|    s2| 2016|   01| 01| 30.1|
|    s1| 2016|   01| 01| 20.5|
|    s2| 2016|   01| 02| 20.4|
+------+-----+-----+---+-----+



In [29]:
score=score_sorted.select("score")
score.show()

+-----+
|score|
+-----+
| 60.2|
| 55.5|
| 52.5|
| 30.1|
| 20.5|
| 20.4|
+-----+



In [30]:
top_3=spark.createDataFrame(score.take(3))
top_3.show()

+-----+
|score|
+-----+
| 60.2|
| 55.5|
| 52.5|
+-----+



In [31]:
top_3.write.mode("overwrite").csv("./output",header=True)